In [193]:
import pandas as pd
from pymongo import MongoClient
from pandas.io.json import json_normalize

client = MongoClient('mongodb://localhost:27017/')
db = client.companies



In [107]:
#Starting consuting time on Mongo DB

one_office = db.companies.find({    #Companies with at least 1 office
    "offices":{
        "$not":{
            "$size":0
        }
    }
},{"name":1, "offices":1})

one_office_df = pd.DataFrame(one_office)
#display(one_office_df)
print(one_office_df.shape)
one_office_df.columns

# With this query we know that there are at least 13,744 registers with offices

(13744, 3)


Index(['_id', 'name', 'offices'], dtype='object')

In [27]:
# We have some criteria when it comes to get the data from our database. The minimums are:
# - There must be some nearby companies that also do design.
# - Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
# - Developers like to be near successful tech startups with that have raised at least 1 Million dollars.
# - Account managers need to travel a lot


In [199]:
# Filtering with mongo
# Therefore, we ask mongodb (politetly) about some of the criteria we are considering.
 
mongodata = db.companies.find( {"$and":[
                                {"offices.latitude": {"$exists": True}},
                                {"offices.latitude": {"$ne": None}},   
                                {"offices.longitude": {"$exists": True}},
                                {"offices.longitude": {"$ne": None}},
                                {"category_code": {"$exists": True}},
                                {"category_code": {"$ne": None}},   
                                {"founded_year": {"$exists": True}}, 
                                {"founded_year": {"$gte": 2003}},
                                {"deadpooled_year": None},
                                {"number_of_employees": {"$exists": True}},
                                {"total_money_raised": {"$exists": True}},
                                {"total_money_raised":{"$ne":None}},
                                {"total_money_raised": {"$not":{"$size":0}}}, 
                                {"$or": [
                                {"total_money_raised": {"$gte": 1000000}},
                                {"category_code":  "design" } ,
                                {"category_code":  "nanotech" } , 
                                {"category_code":  "web" } , 
                                {"category_code":  "software" } , 
                                {"category_code":  "games_video" } , 
                                {"category_code":  "mobile" } , 
                                {"category_code":  "ecommerce" } ,
                                {"category_code":  "advertising" } ,
                                {"category_code":  "enterprise" } ,   
                                {"category_code":  "analytics" } ,
                                {'category_code':'search'},
                                {'category_code':'network_hosting'} ,   
                                {"category_code":  "photo_video" } ,   
                                {"category_code":  "biotech" } ]} ,
                                      
                                ]
                                },
                                {
                               "_id": 0, "category_code": 1,"founded_year": 1, "name": 1, 
                               "offices.country_code": 1, "offices.latitude": 1, "offices.longitude": 1, 
                                "total_money_raised": 1, "number_of_employees":1   
                                }
                            )




In [200]:
#df = pd.DataFrame(mongodata)
#display(df.head())
#print(df["offices"][0])
#print(df["name"][0])

In [124]:
# However, the serie "offices" it's a list with embed dicts. The serie contains information about location and number
# of offices.
# We use json_normalize to extract this information

In [201]:
ydataframe = json_normalize(data = mongodata, record_path = "offices", 
                             meta = ["category_code", "founded_year", "name",
                                    "total_money_raised", "number_of_employees"])

In [202]:
display(ydataframe.head())
print(ydataframe.shape)
print(ydataframe.columns)

,country_code,latitude,longitude,category_code,founded_year,name,total_money_raised,number_of_employees
0,USA,34.090368,-118.393064,web,2006,Geni,$16.5M,18
1,USA,37.484130,-122.169472,web,2003,Gizmoz,$18.1M,None
2,USA,34.057498,-118.446596,mobile,2005,Helio,$0,None
3,USA,40.746497,-74.009447,games_video,2006,Joost,$45M,0
4,USA,37.778613,-122.395289,search,2006,Powerset,$22.5M,60


(4207, 8)
Index(['country_code', 'latitude', 'longitude', 'category_code',
       'founded_year', 'name', 'total_money_raised', 'number_of_employees'],
      dtype='object')


In [212]:
df = ydataframe.reindex(columns=['country_code', 'name', 'founded_year', 'category_code', 'latitude', 'longitude', 
                                 'total_money_raised', 'number_of_employees'])

In [213]:
display(df.head())

,country_code,name,founded_year,category_code,latitude,longitude,total_money_raised,number_of_employees
0,USA,Geni,2006,web,34.090368,-118.393064,$16.5M,18
1,USA,Gizmoz,2003,web,37.484130,-122.169472,$18.1M,None
2,USA,Helio,2005,mobile,34.057498,-118.446596,$0,None
3,USA,Joost,2006,games_video,40.746497,-74.009447,$45M,0
4,USA,Powerset,2006,search,37.778613,-122.395289,$22.5M,60


In [214]:
df["total_offices"] = df["name"].map(df["name"].value_counts())

In [215]:
display(df.head())
print(df.shape)
# On this stage, we've disclouse all the data inside "offices" and added another column with the number of offices for 
# each company. We have the coordinates of each office but need to proceed to generate geopoints.

,country_code,name,founded_year,category_code,latitude,longitude,total_money_raised,number_of_employees,total_offices
0,USA,Geni,2006,web,34.090368,-118.393064,$16.5M,18,1
1,USA,Gizmoz,2003,web,37.484130,-122.169472,$18.1M,None,1
2,USA,Helio,2005,mobile,34.057498,-118.446596,$0,None,1
3,USA,Joost,2006,games_video,40.746497,-74.009447,$45M,0,1
4,USA,Powerset,2006,search,37.778613,-122.395289,$22.5M,60,1


(4207, 9)


In [216]:

def get_offices_locat2(lon, lat):
    print(lon, lat)
    #for data["latitude"] & data["longitude"]:
    offices_locat = {
            "type":"Point",
            "coordinates":[lon, lat]
        }
    return offices_locat

In [217]:
#df.apply(lambda x: get_offices_locat2(x['longitude'], x['latitude']), axis=1)
# With this function we apply the points coordinate conversion

In [218]:
df["geoloc"] = df.apply(lambda x: get_offices_locat2(x['longitude'], x['latitude']), axis=1)


-118.393064 34.090368
-122.169472 37.48413
-118.446596 34.057498
-74.0094471 40.7464969
-122.395289 37.778613
-118.487267 34.017606
-117.20834 32.902266
-122.409173 37.788482
-122.173887 37.480999
-122.0899512 37.4233896
34.851612 31.046051
-73.995625 40.726155
-74.3372 40.801358
-73.99873 40.722655
-122.426804 37.627971
-73.563878 41.049051
-121.932202 37.465645
-122.357242 47.617682
-122.397224 37.762541
-119.306607 37.269175
-122.347533 47.620716
-122.217964 37.488336
-122.302262 37.559287
-119.306607 37.269175
-122.388243 37.757758
-71.118487 42.375392
-122.396744 37.780134
-122.393913 37.780716
-122.137444 37.421621
-118.756618 34.155211
-122.395089 37.793819
4.8948623 52.3745228
-1.3610845 50.9444598
-122.402195 37.786183
-122.02421 37.36965
-122.359397 47.620982
-118.495155 34.0126379
-122.414689 37.758113
-73.9843328 40.7516611
-118.468707 33.991101
-122.4016144 37.7891086
-0.1356235 51.5094731
-122.4058887 37.7791761
-122.206893 37.484619
-122.401116 37.782103
-122.025485 37.4

-73.96831 40.794985
-97.807032 30.414146
-122.406912 37.781002
24.704663 60.214102
-122.1514101 37.4784804
-73.889492 40.646166
-118.13461 34.091562
151.216427 -33.884685
-122.307887 37.54763
-122.211435 37.503147
5.929313 43.124351
-122.099577 37.431769
-122.160671 37.445326
-122.393323 37.783151
-73.985506 40.757929
-74.011827 40.706527
-71.408718 41.826956
-122.075413 37.419156
-122.266978 37.80294
-121.04442 38.673971
-16.9147342 32.6460331
-79.915015 40.431653
-73.7258199 40.8587524
-84.227415 43.626128
-73.981534 40.738832
-110.9864294 32.2745229
-95.47728 30.054922
13.441005 52.502633
-79.388279 43.666953
153.088303 -26.665908
-51.92528 -14.235004
-73.982607 40.74632
-96.714066 32.974194
-89.400702 43.063193
-73.985506 40.757929
12.507358 41.980591
-122.4025016 37.7809512
-79.452609 43.70391
-0.474933 46.330771
-8.7542918 51.7335021
-71.683502 42.037302
-71.316744 42.894414
-95.712891 37.09024
-80.190262 25.774252
-0.011154 51.5320478
-87.657354 41.88583
-73.985878 40.739794
-12

-122.419204 37.775196
-78.744343 42.940938
-117.729229 33.887112
-73.522883 45.523535
-75.377916 40.043029
-122.395708 37.783383
-118.264858 34.05285
-0.147646 51.471958
-122.4438929 37.7717185
144.962991 -37.817748
-75.179869 39.9512991
-122.477959 37.814202
-77.1008 38.9806
7.664067 48.491885
-73.992152 40.754698
-119.306607 37.269175
-3.70325 40.416741
-97.703656 30.387069
2.350987 48.856667
-118.3952969 33.9192583
-6.267494 53.344104
-73.985506 40.757929
-0.208979 51.51819
34.851612 31.046051
16.355 48.220357
133.775136 -25.274398
11.986383 57.696981
34.851612 31.046051
-73.985506 40.757929
30.23279 60.022133
-75.197184 39.964046
-4.271165 55.862878
-122.278022 47.683612
-110.958319 32.240301
-73.998208 40.725568
-122.397884 37.787204
-96.823962 33.075067
-97.735768 30.396989
-73.925194 41.378031
34.7595 32.0554
-73.990076 40.732525
-95.417654 29.760855
-96.662494 33.17514
-122.419204 37.775196
-73.993274 40.743563
-95.5077 29.717624
12.461425 41.823172
-121.96254 37.23697
74.84256

-73.537671 41.058558
4.475797 51.916547
-122.326141 37.536394
-105.008551 39.755363
-79.411513 43.769574
-84.380568 33.852384
2.3249952 48.8785721
-7.6332257 33.5922469
-2.26681 52.232349
8.527005 47.377663
-80.147944 26.02617
4.8926529 52.3644743
-79.458419 43.70452
-122.080843 37.390918
174.774093 -41.2902
-77.334293 38.942964
-0.124569 51.51299
-122.135282 47.582658
-80.235886 25.726688
121.475 31.222222
-77.342896 38.947914
-122.393528 37.781021
-73.580424 45.502556
-80.3817739 26.1223623
13.37402 52.537588
-73.98458 40.74385
-122.035963 37.3725243
-111.8963228 33.555878
-77.156823 39.083916
-122.145153 37.424614
14.362311 46.771087
-1.313068 51.417565
-118.152249 34.633698
-122.295697 47.810914
-122.5162892 37.9260091
-70.7525299 43.2319222
-87.625271 41.864917
80.2285563 13.0650925
-105.13114 39.97764
-77.028561 38.897976
-122.407709 37.781754
-0.698361 52.472029
-122.145038 47.580235
-122.418642 37.758062
-71.325869 42.616152
-118.462421 33.989029
-83.045913 42.330165
-122.69265

27.207361 38.462306
-122.421406 37.781386
-80.3142203 25.8666423
-87.5962319 41.7894201
-121.958687 37.0391044
77.5922642 12.9561278
-73.988071 40.744549
7.6267552 51.9630088
-79.3956689 43.6453258
-122.029217 37.377787
-122.0792041 37.3933789
-123.138565 49.263588
151.1936039 -33.8719926
-74.003466 40.740837
-117.747055 33.65042
-118.864513 34.267061
-3.1577774 55.9735517
-84.368975 33.961646
-74.9822149 44.6686534
-122.4721542 45.6667514
-122.212095 37.485824
12.5605372 55.6739617
-71.590103 42.3389044
-84.389663 33.754487
-71.2252014 42.3127142
-121.1268303 38.6456907
12.0198938 57.7209005
-121.987661 37.386774
4.401195 50.7963208
-82.490137 28.4370407
-122.3326145 47.6024156
8.8088823 53.0785928
-80.2499858 43.5468152
-71.237976 42.430844
-84.058708 34.070164
151.207114 -33.867139
13.261829 47.86582
-85.6734906 42.9608776
-111.8967224 40.4273613
-121.481826 38.517538
77.5384384 12.9737322
-77.367981 38.872668
-71.128231 42.393551
8.0493789 47.394253
-117.2398594 32.9388917
2.359384

-84.361865 34.030223
80.182297 13.0985851
77.5655466 13.0056401
151.19552 -33.881451
-71.24756 42.318138
-122.422503 37.786317
-84.44041 33.909439
13.3740493 52.5375697
-80.144225 25.7903624
-122.402667 37.793473
-71.1469003 42.6751888
-95.452352 29.717881
-0.1010535 51.5232255
-74.1260363 40.9416904
-82.3996325 34.849294
-116.7732162 33.7836802
76.2594981 9.9396253
-118.378229 46.0439
-93.2038492 44.8067033
-73.410677 40.803329
0.1380852 52.1992891
-81.5257116 41.4865099
-71.0213788 42.3983662
2.163444 41.4322413
-86.145459 39.87335
-78.7252479 35.7892464
-73.985273 40.750597
-122.403755 37.796216
-122.947248 49.229195
-80.010645 40.532969
-75.2850602 40.0751561
-87.342177 41.5911112
30.503828 50.453629
-0.1262241 50.8189967
-79.524848 43.78741
13.2170029 55.714683
-86.7889469 36.1749152
-122.408914 37.777995
-122.3050823 37.515853
-118.2551 34.1527593
-77.7514 43.17742
8.5093142 48.8838843
-97.777273 30.498112
-71.1986329 42.4871509
-122.333359 47.603364
7.6341781 51.9266839
-71.2721

4.9549211 52.2986042
2.3509871 48.8566667
9.9819779 53.5688101
-78.7412642 35.7522668
-117.7256083 33.959453
-122.300246 37.553743
-122.347276 47.620973
-71.2459523 42.4236783
18.0644881 59.3327881
12.3959182 51.3210042
11.53451 48.1193794
-82.4431675 38.4208384
22.2930547 60.4500359
-122.419204 37.775196
2.1920231 41.4063346
-80.0520862 26.706239
-0.146659 51.4940757
-85.5796432 42.2927344
-74.032177 40.726517
-73.9952442 40.751234
-73.986951 40.756054
-118.243425 34.052187
-80.001933 40.438423
-0.172364 51.543337
-71.0814466 42.3672873
1.3168395 47.5974286
16.8489579 52.298342
77.5857168 12.910491
-83.7422325 42.2802707
-97.7389543 30.2961708
25.014409 60.2268115
-122.087208 37.349392
-58.3716806 -34.6175545
-118.153136 34.146436
-76.581864 39.281734
-3.9980485 40.6367104
-111.941542 33.519503
-122.419209 37.775206
-122.9145936 49.2608279
-117.86595 33.658546
-122.169472 37.48413
-0.3574186 39.4862314
-81.693716 41.499713
77.6101165 12.9164537
13.3918223 52.5235679
-122.3966273 37.77

In [219]:
display(df.head())
print(df.shape)

,country_code,name,founded_year,category_code,latitude,longitude,total_money_raised,number_of_employees,total_offices,geoloc
0,USA,Geni,2006,web,34.090368,-118.393064,$16.5M,18,1,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,USA,Gizmoz,2003,web,37.484130,-122.169472,$18.1M,None,1,"{'type': 'Point', 'coordinates': [-122.169472,..."
2,USA,Helio,2005,mobile,34.057498,-118.446596,$0,None,1,"{'type': 'Point', 'coordinates': [-118.446596,..."
3,USA,Joost,2006,games_video,40.746497,-74.009447,$45M,0,1,"{'type': 'Point', 'coordinates': [-74.0094471,..."
4,USA,Powerset,2006,search,37.778613,-122.395289,$22.5M,60,1,"{'type': 'Point', 'coordinates': [-122.395289,..."


(4207, 10)


In [224]:
df["number_of_employees"].fillna(0, inplace = True)
df["number_of_employees"] =  df["number_of_employees"].fillna(0.0).astype(int)
print(df.shape)

(4207, 10)


In [225]:
df = df[df["number_of_employees"]>50]


In [227]:
df = df[df.total_money_raised != "$0"]
df.shape

(239, 10)

In [228]:
# With this query we check the most repeated categories for our set of companies (those we previosly selected according
# to our prior specifications)
df["category_code"].value_counts()

software           51
advertising        49
enterprise         40
web                35
games_video        15
ecommerce          11
analytics          11
network_hosting     8
mobile              8
search              6
design              3
photo_video         1
biotech             1
Name: category_code, dtype: int64

In [229]:
df["total_money_raised"].value_counts()

$248M     9
$11M      6
$32M      6
$40M      6
€15.9M    5
$52M      4
$13.3M    4
$40.8M    4
$9.91M    4
$35M      4
$17.1M    3
$33M      3
$84.3M    3
$38.4M    3
$5M       3
$10M      3
$55.6M    3
$4M       3
$37.8M    3
$45M      3
$16.3M    3
$175M     2
$21.8M    2
$131M     2
$3M       2
$34.7M    2
$22.1M    2
$28.4M    2
$100M     2
$28.5M    2
         ..
$61M      1
€25M      1
$44.8M    1
$74.7M    1
$2.5M     1
$20.3M    1
$29M      1
$16M      1
$19.4M    1
$18.3M    1
$2.8M     1
$38M      1
$53.2M    1
$41.3M    1
$136M     1
$39.9M    1
$107M     1
$140M     1
$15M      1
$32.4M    1
$30.3M    1
$84.5M    1
$128M     1
$161M     1
$3.1M     1
$18.4M    1
$3.8M     1
$108M     1
$71.2M    1
$15.5M    1
Name: total_money_raised, Length: 160, dtype: int64

In [230]:
"""def filtering_capital (dframe):
    for letter in dframe:
        if letter[-1]=='k':  
        elif letter[-1]=='K':
        elif letter[-1]=='M':
        elif letter[-1]=='B':
    return dframe
"""





"""

def capital_search(string):
    if string.startswith("$"):
        string=string.replace("$","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= float(string)*1000000
        elif string.endswith("k"):
            string=string.replace("k","")
            string= float(string)*1000
        elif string.endswith("B"):
            string=string.replace("B","")
            string= float(string)*1000000000
        else:
            string=float(string)
    return string         
"""
def capital_search (m):
    money = { # We are only considering the money ponderated in $. We'll filter amount-wise.
        "$":1, "M":2, "k":3, "K":4, "B":5
    }
    for key, numero in money.items(): 
        if key in m: 
            return m
    return None


df["total_money_raised"] = df["total_money_raised"].apply(capital_search)
df["total_money_raised"].value_counts().tail()


$18.4M    1
$3.8M     1
$108M     1
$71.2M    1
$15.5M    1
Name: total_money_raised, dtype: int64

In [231]:
df.shape

(239, 10)

In [232]:
df["total_money_raised"].value_counts

<bound method IndexOpsMixin.value_counts of 4       $22.5M
9         $33M
11      $14.7M
33        $90M
38        $32M
39        $32M
40        $32M
41        $32M
43       $136M
54       $140M
58       $409M
59        $66M
60       $110M
66      $31.2M
68      $22.3M
69        $44M
75      $7.18M
77      $55.6M
78      $55.6M
79      $55.6M
81       $131M
83        $52M
108       $20M
113     $82.7M
119       $29M
139      $670k
141     $40.8M
146     $18.3M
150     $11.5M
160     $39.3M
         ...  
3348      $23M
3460      $13M
3511    $9.95M
3527    $40.8M
3534    $64.2M
3554     $6.5M
3605      $17M
3627      $11M
3628      $11M
3629      $11M
3630      $11M
3631      $11M
3724      $64M
3833     $240M
3845     $3.5M
3846     $3.5M
3859     $3.2M
3890    $39.9M
3907      $45M
3985    $28.1M
4089    $34.7M
4090    $34.7M
4101    $77.5M
4125    $9.91M
4126    $9.91M
4127    $9.91M
4128    $9.91M
4134      £14M
4144       $5M
4161      $35M
Name: total_money_raised, Length: 239, dt

In [236]:
import folium
from folium.plugins import HeatMap

for_map = df.copy()

def hotspots(col_lat, col_lon): #insert values as (df[col_lat], df[col_lon])
    statmad = for_map[['latitude', 'longitude']]
    statmadlist = statmad.as_matrix()
    statmap = folium.Map(zoom_start=11) 
    statmap.add_children(HeatMap(statmadlist, radius=15))
    return statmap    

In [237]:
hotspots(for_map['latitude'], for_map['longitude'])

/home/slimbook/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/slimbook/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  


In [92]:
df.to_dict(orient='records')

[{'country_code': 'USA',
  'name': 'Wetpaint',
  'founded_year': 2005,
  'category_code': 'web',
  'latitude': 47.603122,
  'longitude': -122.333253,
  'total_money_raised': '$39.8M',
  'total_offices': 2,
  'geoloc': {'type': 'Point', 'coordinates': [-122.333253, 47.603122]}},
 {'country_code': 'USA',
  'name': 'Wetpaint',
  'founded_year': 2005,
  'category_code': 'web',
  'latitude': 40.7237306,
  'longitude': -73.9964312,
  'total_money_raised': '$39.8M',
  'total_offices': 2,
  'geoloc': {'type': 'Point', 'coordinates': [-73.9964312, 40.7237306]}},
 {'country_code': 'USA',
  'name': 'Zoho',
  'founded_year': 2005,
  'category_code': 'software',
  'latitude': 37.692934,
  'longitude': -121.904945,
  'total_money_raised': '$0',
  'total_offices': 1,
  'geoloc': {'type': 'Point', 'coordinates': [-121.904945, 37.692934]}},
 {'country_code': 'USA',
  'name': 'Geni',
  'founded_year': 2006,
  'category_code': 'web',
  'latitude': 34.090368,
  'longitude': -118.393064,
  'total_money_rai

In [238]:
df.to_json('afines.json', orient="records")